# Calculating Drivetimes using  OpenRouteService

## Import Required Libraries and Set Working Directory

In [1]:
import openrouteservice
import os 
import getpass
import pandas as pd 
import time
import logging # getting annoying logging message when creating table, turning it off 
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')


In [2]:
#sign up and request free api key from https://openrouteservice.org/

## Import Addresses to Dataframe

Read to DF

In [10]:
df = pd.read_excel('Test_Data1.xlsx',keep_default_na=False)

## Connect to Openrouteservice and Run 
For more information on openrouteservice and how to get an API Key refer to Readme
Connect using API Key

In [ ]:
client = openrouteservice.Client(key='YOURAPIKEYHERE')

Openrouteservice has a 24 hour limit of 2k requests, so if we have more than 2k rows, we'll need to flag them. If this is the second pass at the DF, we want to skip adding these columns, so we don't overwrite them.

In [ ]:
if 'Complete' not in df.columns:
    df['Complete'] = 0 
if 'Time' not in df.columns:
    df['Time'] = 0 

## Pass lat and longs through Openrouteservice using client.direction() to retrieve drivetime in seconds.
May want to cod in skips for NULL lat, longs as each pass will count as a request. The following code passes through each row of the dataframe and runs client.direction() if the row is not complete. If an error related to quota or rate limits is retrieved, the loop exits, and if NULL or invalid coordinates are passed to the API, the loop will skip. 

The below example was utilizing a second pass through after a 24hr limit, so first 2k rows are skipped.

Note: This is just my n00b method of doing this, I'm sure there are different or more streamlined approaches that can be utilized.

In [12]:
for i, row in df.iterrows():
    try:
        if row['Complete']==1:
            print(f'Skipping...{i}') 
        if row['Complete']==0:
            coords = ((row['fnl_long'],row['fnl_lat']),(row['long'],row['lat']))
            res = client.directions(coords)
            secs = res['routes'][0]['summary']['duration']
            time.sleep(1.6) # max is 40 requests per minute..so 1.5 would be max
            df['Time'][i]=secs
            df['Complete'][i]=1
            print(f'Added...{i}')
    except Exception as e:
        result = str(e)
        # 403 ({'error': 'Quota exceeded'})
        # 429 ({'error': 'Rate limit exceeded'})
        if (result[slice(0,3)]=='403') or (result[slice(0,3)]=='429'):
            print('Query limit reached...')
            break
        else:
            print(f'Invalid coordinates...{i}') 
            pass


Skipping...1999
Added...2000
Added...2001
Added...2002
Added...2003
Invalid coordinates...2004
Added...2005
Added...2006
Added...2007
Added...2008
Added...2009
Added...2010
Added...2011
Added...2012
Added...2013
Added...2014
Added...2015
Added...2016
Added...2017
Invalid coordinates...2018
Added...2019
Added...2020
Invalid coordinates...2021
Added...2022
Invalid coordinates...2023
Added...2024
Added...2025
Added...2026
Added...2027
Added...2028
Added...2029
Added...2030
Added...2031
Added...2032
Added...2033
Added...2034
Added...2035
Added...2036
Invalid coordinates...2037
Invalid coordinates...2038
Invalid coordinates...2039
Added...2040
Added...2041
Invalid coordinates...2042
Added...2043
Added...2044
Added...2045
Added...2046
Added...2047
Invalid coordinates...2048
Added...2049
Added...2050
Note: Deleted all print outputs before row 1999 and after row 2050 (took up too many lines in guide, this is just to show that the loop handles results)


## Save Results to Excel

In [13]:
df.to_excel('Test_Data2.xlsx') 